![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Large Language Models und Transformer
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>Umgebung einrichten</font> </br></p>
!uv pip install --system --prerelease allow -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
install_packages(['pypdf','pdfkit', 'markdown2', 'weasyprint'])

# 1 | Large Language Models
---

LLMs funktionieren im Wesentlichen durch die *Vorhersage des wahrscheinlichsten nächsten Worte*s in einem gegebenen Text, basierend auf einer riesigen Menge an Trainingsdaten aus dem Internet. Dieser Prozess wird durch Billiarden von Parametern gesteuert, die während des Trainings optimiert werden. Ein entscheidender Aspekt ist die Architektur von **Transfomern**, die es LLMs erlaubt, den gesamten Text **parallel** zu verarbeiten, anstatt Wort für Wort, und dabei den Kontext durch einen Mechanismus namens **Self Attention** zu berücksichtigen. Das Ergebnis ist ein System, das erstaunlich flüssige und fast immer Texte generieren kann, obwohl das genaue *Warum* hinter den Vorhersagen aufgrund der Komplexität und Größe des Modells schwer zu verstehen ist.


<p><font black='red' size="5">
Top 5 der Modellansätze:
</font></p>

| Ansatz                                                         | Kurzbeschreibung                                                            | Einsatzgebiet(e)                                   | Beispiele                          |
| -------------------------------------------------------------- | --------------------------------------------------------------------------- | -------------------------------------------------- | ---------------------------------- |
| **1. Autoregressive Modelle**                                  | Token-weises Vorhersagemodell auf Basis vorheriger Tokens                   | Textgenerierung, Dialogsysteme, Code               | GPT, LLaMA, Mistral                |
| **2. Autoencoder (z. B. Variational AE)**                      | Komprimieren und Rekonstruieren von Daten durch Encoder-Decoder-Architektur | Bild-, Audioverarbeitung, latente Repräsentationen | BERT (teilweise), VQ-VAE           |
| **3. Diffusionsmodelle**                                       | Lernen der Rückführung von Rauschen zu Daten durch viele rekursive Schritte | Bild-, Audio-, Video- und Textgenerierung          | DALL·E 2, Stable Diffusion, Imagen |
| **4. RAG (Retriever-Augmented Generation)**                    | Kombination aus LLM und externem Dokument-Retrieval                         | Q\&A, Recherche, domänenspezifische Generierung    | GPT+ChromaDB, LangChain-RAG        |
| **5. Reinforcement Learning mit menschlichem Feedback (RLHF)** | Nachträgliche Optimierung generativer Modelle auf menschliche Präferenzen   | Feinjustierung von Chatbots, ethisches Verhalten   | GPT-4 (Fine-Tuning), Claude        |


**Demo Arbeitsweise:**    
[Autoregressive Modelle](https://poloclub.github.io/transformer-explainer/)    
[Diffusionmodel](https://huggingface.co/spaces/multimodalart/Dream)

<p><font color='black' size="5">
Merkmale von LLMs
</font></p>


<p><font color='blue' size="4">
Offene vs. geschlossene Modelle
</font></p>

**Offene Modelle** (Open Source, offenes System),  bedeuten, dass die trainierten Parameter eines Modells für die Öffentlichkeit zugänglich sind. Dadurch haben Wissenschaftler und Entwickler die Möglichkeit, die Mechanismen des Modells nachzuvollziehen, Forschungsarbeiten zu reproduzieren sowie Anpassungen oder Verbesserungen vorzunehmen.   
**Geschlossene Modelle** (Closed Source, proprietäres System) hingegen sind Modelle, bei denen der Zugriff auf die Parameter eingeschränkt ist. Unternehmen setzen diese oft in kommerziellen Produkten oder Dienstleistungen ein, da die Veröffentlichung der Gewichte geschäftliche Interessen oder den Schutz der Nutzerdaten beeinträchtigen könnt



<p><font color='blue' size="4">
Parametergewichte
</font></p>

Die Anzahl der Parametergewichte eines Modells gibt Aufschluss über dessen Kapazität und Komplexität. Sie wird typischerweise in Millionen (M), Milliarden (B) oder Billionen (T) angegeben. Eine höhere Parameteranzahl erhöht grundsätzlich die Fähigkeit des Modells, komplexe Zusammenhänge und feine Unterschiede in den Daten zu erkennen. Allerdings ist dies kein eindeutiger Indikator für die Leistungsfähigkeit in jeder Anwendung, sondern eher ein Hinweis auf das allgemeine Potenzial des Modells.  


**Vorteile:**  
* **Größere Lernfähigkeit:** Modelle mit mehr Parametern können feinere und differenziertere Muster in Daten erfassen, was ihre Präzision und Wirksamkeit bei verschiedenen Aufgaben steigern kann.  
* **Bessere Anpassungsfähigkeit:** Umfangreichere Modelle haben oft eine stärkere Fähigkeit zur Generalisierung und können unter geeigneten Trainingsbedingungen auch auf neue, unbekannte Daten besser reagieren.  

**Nachteile:**  
* **Hoher Rechenaufwand:** Eine größere Anzahl an Parametern erfordert mehr Rechenleistung für Training und Inferenz, was leistungsfähige Hardware und längere Verarbeitungszeiten nötig macht.  
* **Gefahr der Überanpassung:** Ohne angemessene Regularisierungstechniken besteht die Gefahr, dass das Modell sich zu stark an die Trainingsdaten anpasst und bei neuen, variierenden Datensätzen schlechter abschneidet.  
* **Umweltbelastung:** Das Training umfangreicher Modelle verbraucht erheblich mehr Energie, was zu einem höheren CO₂-Ausstoß führt.



<p><font color='blue' size="4">
Kontextfenstergröße
</font></p>

Die Kontextfenstergröße eines Modells gibt an, wie viele Token (Wörter oder Wortbestandteile) es bei der Generierung oder Vorhersage von Text gleichzeitig verarbeiten kann. Diese Eigenschaft ist aus mehreren Gründen essenziell:

+ **Erweiterter Kontext**: Ein größeres Kontextfenster erlaubt es dem Modell, mehr Informationen zu berücksichtigen, was zu kohärenteren und inhaltlich präziseren Ergebnissen führt. Dies ist besonders vorteilhaft für Aufgaben, die lange Texte oder komplexe Zusammenhänge erfordern.
+ **Erfassung von Abhängigkeiten**: Ein Modell mit größerem Kontextfenster kann weiter zurückliegende Zusammenhänge im Text besser erfassen, was seine Leistungsfähigkeit bei Aufgaben wie Textzusammenfassungen, Frage-Antwort-Systemen oder interaktiven Dialogen erheblich steigert.

Ein fundiertes Verständnis dieser Faktoren hilft bei der Auswahl eines geeigneten Modells für spezifische Anwendungsfälle und ermöglicht eine gezielte Optimierung der Leistung.



<p><font color='blue' size="4">
Token
</font></p>

In Large Language Models (LLMs) wie GPT (Generative Pre-trained Transformer) stellen Token die grundlegenden Einheiten des verarbeiteten Textes dar. Ein **Token** kann ein vollständiges Wort, ein Wortbestandteil oder ein einzelnes Zeichen sein. Wie genau ein Token definiert wird, hängt vom verwendeten Tokenizer ab, der während des Modelltrainings zum Einsatz kam. So könnte das Wort *Hausboot* entweder als einzelnes Token betrachtet oder in die Bestandteile *Haus* und *boot* zerlegt werden - abhängig von der jeweiligen Tokenisierungsstrategie.  

Die Nutzungskosten eines LLMs zur Textgenerierung werden in der Regel anhand der verarbeiteten Token berechnet. Dabei zählen sowohl die Token aus der Eingabe als auch diejenigen, die das Modell als Antwort generiert. Da die Verarbeitung jedes Tokens insbesondere bei sehr großen Modellen mit Milliarden von Parametern erhebliche Rechenressourcen erfordert, beeinflusst die Anzahl der Token direkt die Rechenkosten. Daher ist ein effizientes Management der Token-Nutzung essenziell, um Kosten zu optimieren.  

Die **Kontextfenstergröße** eines LLM gibt an, wie viele Token aus einer Eingabe das Modell gleichzeitig berücksichtigen kann. Hat ein Modell beispielsweise ein Kontextfenster von 1.024 Token, kann es nur die letzten 1.024 Token eines Textes für die nächste Vorhersage verwenden. Ist der Eingabetext länger, kann das Modell auf frühere Abschnitte nicht mehr direkt zugreifen, was sich auf die Kohärenz und Relevanz der generierten Antworten auswirken kann.



<p><font color='blue' size="4">
Temperatur
</font></p>

Bei LLM wie GPT bezieht sich OpenAI mit dem Begriff **Temperatureinstellung** auf die Steuerung der **Zufälligkeit** oder **Kreativität** der generierten Antworten. Eine niedrigere Temperatur (z. B. 0,0) sorgt für **deterministische** und vorhersehbare Ergebnisse, da das Modell vorrangig die wahrscheinlichsten Antworten wählt. Eine höhere Temperatur (z. B. 1,0) hingegen führt zu einer größeren Varianz und kann **kreativere**, aber auch weniger vorhersehbare Antworten erzeugen. Der Wertebereich variiert je nach Anwendung oder Benutzeroberfläche, liegt jedoch typischerweise zwischen 0 und 1. Durch die Anpassung der Temperatur lässt sich das Modell gezielt steuern, um ein passendes Verhältnis zwischen Konsistenz und Originalität für unterschiedliche Aufgaben oder Nutzerpräferenzen zu erreichen.


<p><font color='blue' size="4">
Große Sprachmodelle
</font></p>

Die folgende Tabelle gibt einen Überblickt für bekannte LLMs (Stand: 04.2025)

| Name                | Ersteller  | Open/Closed | Input-Token | Output-Token | Anzahl Parameter      |
| :------------------ | :--------- | :---------- | :---------- | :----------- | :-------------------- |
| GPT-4.1             | OpenAI     | Closed      | 1M          | 32K          | Unbekannt             |
| GPT-4.1 mini        | OpenAI     | Closed      | 1M          | 32K          | Unbekannt             |
| GPT-4.1 nano        | OpenAI     | Closed      | 1M          | 32K          | Unbekannt             |
| GPT-4.5             | OpenAI     | Closed      | 128K        | 4K           | Unbekannt             |
| GPT-4o              | OpenAI     | Closed      | 128K        | 4K           | ~1.8T (geschätzt)     |
| GPT-4o mini         | OpenAI     | Closed      | 128K        | 4K           | ~8B (geschätzt)       |
| Claude 3.7 Sonnet   | Anthropic  | Closed      | 200K        | 4K           | >200B (geschätzt)     |
| Claude 3.5 Sonnet   | Anthropic  | Closed      | 200K        | 4K           | ~175–200B (geschätzt) |
| Claude 3 Opus       | Anthropic  | Closed      | 200K        | 4K           | Unbekannt             |
| Gemini 2.5 Pro Exp. | Google     | Closed      | 1M          | 4K           | Unbekannt             |
| Gemini 2.0 Pro      | Google     | Closed      | 2M          | 4K           | Unbekannt             |
| Gemini 2.0 Flash    | Google     | Closed      | 1M          | 4K           | Unbekannt             |
| Gemini 1.5 Pro      | Google     | Closed      | 2M          | 4K           | ~1.5T (geschätzt)     |
| Gemini 1.5 Flash    | Google     | Closed      | 1M          | 4K           | ~8B (geschätzt)       |
| Llama 3.1 405B      | Meta       | Open        | 512K        | 4K           | 405B                  |
| Llama 3.1 70B       | Meta       | Open        | 256K        | 4K           | 70B                   |
| Llama 3.1 8B        | Meta       | Open        | 128K        | 4K           | 8B                    |
| Mistral Large 2     | Mistral.AI | Open        | 32K         | 4K           | 123B                  |
| Mistral 7B          | Mistral.AI | Open        | 32K         | 4K           | 7.3B                  |
| DeepSeek R1         | DeepSeek   | Open        | 128K        | 4K           | 671B (37B aktiv)      |
| Qwen 2.5-Max        | Alibaba    | Open        | 32K         | 4K           | Unbekannt             |
| Mixtral 8x22B       | Mistral.AI | Open        | 65K         | 4K           | 141B (39B aktiv)      |
| Gemma 2 (9B/27B)    | Google     | Open        | 8K          | 4K           | 9B, 27B               |
| DBRX                | Databricks | Open        | 32K         | 4K           | 132B                  |
| Nemotron-4          | Nvidia     | Open        | 32K         | 4K           | 340B                  |

**Im Kurs verwenden wir i.W.:**  
- **gpt-4o-mini**: Standardmodell für die meisten Aufgaben.  
- **gpt-4o**: Für detailliertere Antworten.  
- **o3-mini**: Optimiert für logische Schlussfolgerungen.
- **dall-e-3**: Standard-Modell für Bildgenerierung

Es können alternativ auch die neuen Modell **gpt-4.1**, **o4-mini** oder **gpt-image-1** eingesetzt werden.

[OpenAI Modelle](https://platform.openai.com/docs/models)

# 2 | Foundation Models
---

Foundation Models sind große KI-Modelle, die auf riesigen Mengen an Daten trainiert werden. Sie bilden eine breite Basis an Wissen und Fähigkeiten, auf der sich spezialisierte Anwendungen aufbauen lassen. Dazu gehören nicht nur große Sprachmodelle (Large Language Models, LLMs) wie GPT, sondern auch Modelle für Bilder, Audio und andere Datentypen. Foundation Models werden oft im sogenannten *self-supervised learning* trainiert: Sie lernen Muster, Strukturen und Zusammenhänge in Daten, ohne dass diese manuell beschriftet sein müssen.

Ein großer Vorteil dieser Modelle ist ihre vielseitige Einsetzbarkeit: Nach dem Training können sie an unterschiedlichste Aufgaben angepasst werden, etwa für Textzusammenfassungen, Bilderkennung oder sogar multimodale Anwendungen, die Text und Bild kombinieren. Dadurch sind Foundation Models zu einem wichtigen Baustein in der aktuellen KI-Entwicklung geworden.

Mehrere führende Technologieunternehmen und Forschungsorganisationen bieten Foundation Models an. Dazu gehören OpenAI mit GPT (Generative Pre-trained Transformer), Google mit BERT (Bidirectional Encoder Representations from Transformers) und Facebook (Meta) mit Modellen wie RoBERTa (Robustly Optimized BERT Pretraining Approach).

Das Training eines Foundation Models ist extrem aufwendig: Es erfordert eine enorme Menge an Daten, die sorgfältig gesammelt, bereinigt und verarbeitet werden müssen. Außerdem sind dafür spezialisierte Hardwaresysteme und sehr viel Rechenleistung notwendig, was die Entwicklung teuer und komplex macht. Für Einzelpersonen oder (kleinere) Organisationen ist es daher oft sinnvoller, bestehende Foundation Models zu verwenden und sie gezielt für eigene Aufgaben zu optimieren (beispielsweise durch *Fine-Tuning*).

In der praktischen Anwendungen geht es in der Regel darum, wie man vorhandene Foundation Models effektiv einsetzt, anpasst und für eigene Projekte nutzt. Denn ein vollständiges Modell selbst zu entwickeln, würde deutlich mehr Zeit und Ressourcen erfordern, als ein Kurs leisten kann.

Trotz ihrer großen Möglichkeiten bergen Foundation Models auch Herausforderungen, zum Beispiel hinsichtlich Verzerrungen in den Trainingsdaten oder unvorhersehbaren Ergebnissen. Ein bewusster und verantwortungsvoller Umgang mit diesen Modellen ist deshalb besonders wichtig.


![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/FoundationModels-klein.png)

# 3 | Aufgaben Sprachverarbeitung
---


Large Language Models (LLMs) wie GPT übernehmen eine Vielzahl von Aufgaben rund um die Verarbeitung natürlicher Sprache. Neben bekannten Anwendungsfeldern wie Textklassifizierung, Textzusammenfassung und Textgenerierung bieten sie auch Unterstützung bei Analyse, Transformation, Strukturierung und Wissensgewinnung aus Texten. Die folgende Übersicht zeigt typische Aufgabenbereiche im Überblick.

| **Kategorie**                        | **Aufgabe**                      | **Beschreibung**                                                                     |
| ------------------------------------ | -------------------------------- | ------------------------------------------------------------------------------------ |
| 🧠 **Analyse & Klassifikation**      | **Textklassifizierung**          | Einordnung eines Textes in vordefinierte Klassen (z. B. Spam/kein Spam).             |
|                                      | Sentimentanalyse                 | Einschätzung der Stimmung (positiv/negativ/neutral).                                 |
|                                      | Themenklassifikation             | Zuordnung von Texten zu Themen oder Kategorien.                                      |
|                                      | **Named Entity Recognition (NER)**   | Erkennung von Namen, Orten, Organisationen usw. im Text.                             |
|                                      | Relationsextraktion              | Erkennen logischer oder semantischer Beziehungen zwischen Entitäten.                 |
|                                      | Bias- oder Hate-Speech-Erkennung | Klassifikation diskriminierender oder voreingenommener Sprache.                      |
| 📝 **Generierung & Transformation**  | **Textgenerierung**              | Erzeugen von Texten auf Basis von Prompts (z. B. Geschichten, Antworten, Code).      |
|                                      | **Textzusammenfassung**          | Komprimierung eines Textes unter Erhalt der Kernaussage (extraktiv oder abstraktiv). |
|                                      | Paraphrasierung                  | Umformulierung bei gleichbleibendem Sinn.                                            |
|                                      | Stiltransfer                     | Umwandlung des Sprachstils (z. B. sachlich ↔ kreativ).                               |
|                                      | Textvereinfachung                | Vereinfachung komplexer Formulierungen.                                              |
|                                      | Sprachübersetzung                | Übersetzung zwischen Sprachen.                                                       |
| 🧩 **Ergänzung & Vervollständigung** | Autovervollständigung            | Vorhersage des nächsten Wortes oder Satzes.                                          |
|                                      | Lückentext-Ergänzung             | Einfügen sinnvoller Wörter/Sätze in Lücken.                                          |
|                                      | **Story- oder Textfortsetzung**      | Kreative Weiterführung eines gegebenen Textes.                                       |
| 📚 **Informationsgewinnung**         | Schlüsselbegriffextraktion       | Extraktion zentraler Begriffe aus einem Text.                                        |
|                                      | Themenextraktion                 | Identifikation der zentralen Themen eines Textes.                                    |
|                                      | Frage-Antwort (QA)               | Beantwortung von Fragen zu einem Text (extraktiv oder generativ).                    |
|                                      | Faktenprüfung                    | Bewertung der Richtigkeit von Aussagen.                                              |
|                                      | **Retrieval-Augmented Generation**   | Kombination von Textgenerierung mit Wissensabfragen aus externen Quellen.            |
| 🛡️ **Sicherheit & Strukturierung**  | Plagiatserkennung                | Erkennung von Ähnlichkeiten mit bestehenden Texten.                                  |
|                                      | Anonymisierung                   | Entfernen oder Maskieren sensibler Daten.                                            |
|                                      | **Gliederungserstellung**            | Erstellung strukturierter Inhaltsverzeichnisse.                                      |
|                                      | Formatumwandlung                 | Konvertierung von Texten in bestimmte Formate (z. B. E-Mail, Bericht, Code).         |


# 4 | Modellaufbau - mit History
---


Im vorliegenden Code wird ein Chatbot-Modell schrittweise aufgebaut. Der Modellaufbau ist modular und kann als `template` für nachfolgende Konversationsmodelle verwendet werden.

Diese strukturierte Aufteilung macht das Modell robust, wartbar und erweiterbar für verschiedene Anwendungsfälle im Bereich der Konversations-KI.



**Erläuterung der externen Funktionen:**

- `from langchain_openai import ChatOpenAI`: Importiert die Klasse `ChatOpenAI`, die zur Integration von OpenAI's Chatbot-Funktionalitäten in die LangChain-Bibliothek genutzt wird.
- `from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder`: Erlaubt die Definition von Vorlagen für Chat-Prompts und Platzhalter für Nachrichten in LangChain.
- `from langchain.schema import HumanMessage, AIMessage, SystemMessage`: Definiert verschiedene Nachrichtentypen in LangChain, die zwischen menschlichen, KI- und Systemnachrichten unterscheiden.
- `from langchain_core.output_parsers.string import StrOutputParser`: Importiert eine Klasse, die zum Parsen von KI-Output in String-Format in der LangChain-Core-Bibliothek verwendet wird.

In [ ]:
# Abschnitt 1: Importe
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
# Abschnitt 2: Konstanten definieren
model_name = "gpt-4o-mini"
temperature = 0.3

system_prompt = """
Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.

Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert

Bei deiner Arbeit:
- Analysierst du Fragen sorgfältig
- Gibst präzise und relevante Antworten
- Erkennst den Kontext der Anfrage
- Bietest bei Bedarf weiterführende Informationen
- Bleibst sachlich und neutral

Formatiere alle Antworten in Markdown für optimale Lesbarkeit.
"""

In [ ]:
# Abschnitt 3: Chat-Komponenten initialisieren

# Die Chat-Prompt definieren mit drei Variablen: system_prompt, history, question
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# LLM definieren
llm = ChatOpenAI(model=model_name, temperature=temperature)

parser = StrOutputParser()

# Die Konversationskette definieren
chain = prompt | llm | parser

In [ ]:
# Abschnitt 4: Funktionen definieren

def interact_with_ai(input, history):
    """Führt eine einzelne Interaktion mit der KI durch."""

    # -- Aufruf der Kette
    response = chain.invoke(
        {"system_prompt": system_prompt, "history": history, "input": input}
    )

    # -- Ausgabe
    mprint("### 🧑‍🦱 Mensch:")
    question = input.replace("\n", "<br>")
    mprint(input)

    mprint("### 🤖 KI:")
    mprint(response)

    # -- Memory-Mangament
    history.extend([HumanMessage(content=question), AIMessage(content=response)])

In [ ]:
# Abschnitt 5: Hauptprogramm

# Historie wird initialisiert
history = [SystemMessage(content=system_prompt)]

# Liste mit user-input
user_input = [
    "Mein Name ist Ralf",
    "Warum ist der Himmel blau?",
    "Und warum ist er manchmal rot?",
    "Wie ist mein Name?"
]

# Chain-Aufruf in einer Schleife
for input in user_input:
    interact_with_ai(input, history)

<p><font color='black' size="5">
Neue Konversation starten
</font></p>

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=system_prompt)]
input = "Erstelle eine Tabelle der fünf bevölkerungsreichsten Ländern mit Bevölkerung und BIP."
interact_with_ai(input, history)

# 5 | Textgenerierung
---

Die Generierung von Text gehört zu den häufigsten Anwendungsfällen für **Large Language Models (LLMs)**. Die Erstellung von natürlichem Text folgen einem einem ähnlichen Prinzip wie zur Codegenerierung.  

Anstatt eine interaktive Chatumgebung zu verwenden, erfolgt die Textgenerierung durch gezielte **Eingabeaufforderungen (Prompts)**, die an **OpenAI** gesendet werden. Das Modell verarbeitet diese Anfragen und liefert darauf basierend den generierten Text.  

<p><font color='black' size="5">
Muster zur Textgenerierung
</font></p>

Bei der grundlegenden Textgenerierung gibt es verschiedene **Prompt-Muster (Prompting-Methoden)**, die sich je nach Detailgrad der bereitgestellten Informationen unterscheiden. Diese Muster beeinflussen, wie das **Large Language Model (LLM)** den gewünschten Text erzeugt.  

Im Folgenden werden verschiedene dieser Muster untersucht und analysiert, wie sie sich auf die Qualität und Struktur der generierten Inhalte auswirken.

* Zero-Shot-Prompt
* One-Shot-Prompt
* Few-Shot-Prompt

<p><font color='black' size="5">
Zero-Shot Textgenerierung
</font></p>

Ein Zero-Shot-Prompt zur Texterstellung ist eine Technik, bei der ein Sprachmodell ohne vorherige Anpassung oder spezielles Training auf eine bestimmte Aufgabe direkt mit einer einzigen Eingabeaufforderung genutzt wird. Um diesen Ansatz erfolgreich anzuwenden, ist es entscheidend, einen präzisen und ausführlichen Prompt zu formulieren, der genau beschreibt, welche Art von Inhalt generiert werden soll. Dabei sollten Stil, Struktur und relevante Details oder Einschränkungen klar angegeben werden. Beispielsweise kann für eine geschäftliche E-Mail der gewünschte Tonfall (formell oder informell), die wesentlichen Inhalte (wie Termin, Thema, Teilnehmer) sowie eine Handlungsaufforderung spezifiziert werden. Da das Modell allein auf die im Prompt enthaltenen Informationen angewiesen ist, sollte das gewünschte Ergebnis möglichst eindeutig formuliert sein. Diese Methode ist flexibel einsetzbar und erlaubt die Erstellung vielfältiger Textarten, ohne dass eine vorherige Modellanpassung erforderlich ist.  

Das folgende Beispiel demonstriert eine Zero-Shot-Eingabeaufforderung: Es werden verschiedene Anforderungen gestellt und Informationen zu einem Studierenden bereitgestellt, jedoch ohne eine explizite Vorlage für das Sprachmodell vorzugeben.

**Es wird der bereits erstellte ChatBot verwendet und leicht angepasst.**

In [ ]:
# Anpassung: Abschnitt 2: Konstanten definieren
system_prompt = """
Du bist ein hilfreicher KI-Assistent, der bei der Erstellung von Empfehlungsschreiben hilft.
"""

In [ ]:
# Abschnitt 5: Hauptprogramm
user_input = """
    Erstellen Sie ein positives Empfehlungsschreiben für Maria Weber, eine meiner Studierenden im Kurs ML 401 an der Technischen Universität München. Mein Name ist Dr. Julia Berger. Sie bewirbt sich für einen Master of Science in Data Science. Geben Sie mir nur den Haupttext des Schreibens, keine Kopf- oder Fußzeile.
    Formatiere alle Antworten in Markdown für optimale Lesbarkeit.

    Unten ist ihre Anfrage:

    Sehr geehrte Frau Dr. Berger,

    ich hoffe, es geht Ihnen gut! Ich bin Maria Weber (Matrikelnr.: 7890), eine Absolventin der TU München, die im Juni 2023 meinen Bachelor in Wirtschaftsmathematik abgeschlossen habe.

    Im Wintersemester 2022/23 hatte ich das Privileg, Ihren Kurs ML 401: Grundlagen des maschinellen Lernens zu besuchen. Der Kurs war ein Wahlpflichtfach in meinem Bachelorprogramm. Die Inhalte haben mich sehr begeistert und ich habe mich durchgehend intensiv eingebracht, was sich in der Note 1,0 widerspiegelte.

    Nach meinem Abschluss mit einer Gesamtnote von 1,1 – der besten Note meines Jahrgangs – arbeite ich als Data Analytics Consultant bei der Firma DataTech GmbH. Meine Hauptaufgabe liegt in der Entwicklung von KI-gestützten Analysetools für Geschäftsprozesse. Um meine Expertise im Bereich Data Science zu vertiefen, möchte ich berufsbegleitend einen Master in Data Science absolvieren. Ich schätze Ihre Forschungsarbeit sehr (ich verfolge regelmäßig Ihre Publikationen und habe sie auch in meinem Team vorgestellt) und Ihre Einschätzung wäre für meine Bewerbung sehr wertvoll.

    Ich bewerbe mich für folgende Programme:
    - ETH Zürich, Master of Science in Data Science
    - TU Berlin, Master of Science in Data Engineering

    Dürfte ich Sie um ein Empfehlungsschreiben für diese Bewerbungen bitten? Meinen Lebenslauf habe ich beigefügt und ich stelle Ihnen gerne weitere Informationen zur Verfügung.

    Vielen Dank für die Berücksichtigung meiner Anfrage.

    Mit freundlichen Grüßen
    Maria Weber
    """

In [ ]:
history = [SystemMessage(content=system_prompt)]
interact_with_ai(user_input, history)

<p><font color='black' size="5">
One-Shot Textgenerierung
</font></p>

Ein One-Shot-Prompt zur Textgenerierung ist eine Methode, bei der ein Sprachmodell mit einer einzigen, detaillierten Eingabe angewiesen wird, auf deren Basis es einen zusammenhängenden Text erstellt. Um diese Technik wirkungsvoll einzusetzen, sollte der Prompt klar und präzise formuliert sein und alle relevanten Informationen sowie den gewünschten Kontext enthalten. Dabei ist es ratsam, Stil, Ton und spezifische inhaltliche Aspekte genau zu definieren.  

Möchte man beispielsweise eine atmosphärische Beschreibung einer Küstenstadt generieren lassen, sollten wesentliche Merkmale wie Tageszeit, Stimmung und bestimmte visuelle oder emotionale Eindrücke explizit benannt werden. Eine solche detaillierte Vorgabe erleichtert es dem Modell, die Anforderungen zu erfassen und passgenaue Inhalte zu erstellen.  

Nachdem der Prompt formuliert wurde, kann er direkt in das Textgenerierungstool eingegeben werden. Falls das Ergebnis nicht vollständig den Erwartungen entspricht, lässt sich der Prompt gezielt anpassen, um die Qualität und Relevanz der Ausgabe weiter zu optimieren.

In [ ]:
# Abschnitt 5: Hauptprogramm
user_input = """
Basierend auf diesem Beispiel, erstelle ein neues akademisches Empfehlungsschreiben.
Behalte den professionellen Ton bei, aber passe die Details an:

BEISPIEL:
"Ich freue mich sehr, Maria Weber für den Master of Science in Data Science zu empfehlen. Als ihre Dozentin im Kurs ML 401: Grundlagen des maschinellen Lernens an der TU München konnte ich ihre außergewöhnlichen akademischen Fähigkeiten beobachten. Frau Weber schloss den anspruchsvollen Kurs mit der Note 1,0 ab und demonstrierte dabei ein tiefgreifendes Verständnis für maschinelles Lernen und dessen praktische Anwendungen.

Besonders beeindruckend war ihre Fähigkeit, komplexe Konzepte nicht nur zu verstehen, sondern auch auf reale Problemstellungen anzuwenden. In ihrer Position als Data Analytics Consultant bei DataTech GmbH setzt sie diese Fähigkeiten erfolgreich in der Entwicklung KI-gestützter Analysetools ein.

Mit ihrem Bachelorabschluss in Wirtschaftsmathematik (Note 1,1) als Jahrgangsbeste hat Frau Weber bereits bewiesen, dass sie höchsten akademischen Anforderungen gerecht wird. Ihre analytischen Fähigkeiten, gepaart mit ihrer Lernbereitschaft und ihrem Engagement, machen sie zu einer idealen Kandidatin für ein weiterführendes Studium im Bereich Data Science.

Aufgrund ihrer bisherigen Leistungen und ihres Potenzials empfehle ich Frau Weber nachdrücklich für den Master of Science in Data Science. Sie wird zweifellos einen wertvollen Beitrag zum Programm leisten."

ANWEISUNGEN:
1. Erstelle ein neues Empfehlungsschreiben mit ähnlicher Struktur
2. Ändere:
- Namen
- Studienfach
- Universitäten
- Kurse
- Noten
- aktuelle Berufstätigkeit
3. Behalte bei:
- Vier-Absatz-Struktur
- Professionellen Ton
- Spezifische Beispiele für Leistungen
- Klare Empfehlung am Ende

GEWÜNSCHTES FORMAT:
- Nur Haupttext
- Keine Anrede/Grußformel
- In Markdown formatiert
"""

In [ ]:
history = [SystemMessage(content=system_prompt)]
interact_with_ai(user_input, history)

<p><font color='black' size="5">
Few-Shot Textgenerierung
</font></p>

Ein Few-Shot-Prompt gibt einem Modell eine begrenzte Anzahl von Beispielen, um dessen Reaktion gezielt zu steuern. Diese Methode eignet sich besonders für Sprach- oder Bildgenerierungsmodelle, da sie dem Modell hilft, Muster oder Stile anhand weniger Eingaben zu erkennen und entsprechend nachzubilden.  

Ein typischer Few-Shot-Prompt für ein Textgenerierungsmodell enthält mehrere Beispielpaare aus Eingaben und den gewünschten Ausgaben. Dadurch erhält das Modell eine Orientierung und kann ähnliche Inhalte mit höherer Präzision erzeugen. Dieser Ansatz ermöglicht eine gezielte Anpassung der Modellantworten, ohne dass eine umfassende Trainingsphase erforderlich ist.  

Few-Shot-Prompting verbessert die Qualität und Konsistenz der generierten Texte und macht das Modell flexibler einsetzbar – insbesondere für Aufgaben, die kreative Feinabstimmung oder spezifische Stilvorgaben erfordern.

In [ ]:
# Abschnitt 5: Hauptprogramm
user_input = """
    Basierend auf diesen Beispielen, erstelle ein neues akademisches Empfehlungsschreiben.
    Behalte den professionellen Ton bei, aber passe die Details an:

    BEISPIEL 1:
    "Ich freue mich sehr, Maria Weber für den Master of Science in Data Science zu empfehlen. Als ihre Dozentin im Kurs ML 401: Grundlagen des maschinellen Lernens an der TU München konnte ich ihre außergewöhnlichen akademischen Fähigkeiten beobachten. Frau Weber schloss den anspruchsvollen Kurs mit der Note 1,0 ab und demonstrierte dabei ein tiefgreifendes Verständnis für maschinelles Lernen und dessen praktische Anwendungen.

    Besonders beeindruckend war ihre Fähigkeit, komplexe Konzepte nicht nur zu verstehen, sondern auch auf reale Problemstellungen anzuwenden. In ihrer Position als Data Analytics Consultant bei DataTech GmbH setzt sie diese Fähigkeiten erfolgreich in der Entwicklung KI-gestützter Analysetools ein.

    Mit ihrem Bachelorabschluss in Wirtschaftsmathematik (Note 1,1) als Jahrgangsbeste hat Frau Weber bereits bewiesen, dass sie höchsten akademischen Anforderungen gerecht wird. Ihre analytischen Fähigkeiten, gepaart mit ihrer Lernbereitschaft und ihrem Engagement, machen sie zu einer idealen Kandidatin für ein weiterführendes Studium im Bereich Data Science.

    Aufgrund ihrer bisherigen Leistungen und ihres Potenzials empfehle ich Frau Weber nachdrücklich für den Master of Science in Data Science. Sie wird zweifellos einen wertvollen Beitrag zum Programm leisten."

    BEISPIEL 2:
    "Mit großer Überzeugung empfehle ich Thomas Bauer für den Master of Science in Robotik. Als sein Betreuer im Kurs ROB 301: Einführung in die Robotik am Karlsruher Institut für Technologie habe ich sein außerordentliches technisches Talent kennengelernt. Herr Bauer erreichte im Kurs die Note 1,3 und zeigte dabei besondere Stärken in der praktischen Roboterprogrammierung.

    Seine innovative Herangehensweise an komplexe Problemstellungen war bemerkenswert. Als Werkstudent bei RoboTech entwickelte er bereits erfolgreich autonome Navigationssysteme für Industrieroboter und bewies damit seine Fähigkeit, theoretisches Wissen praktisch umzusetzen.

    Sein Bachelorabschluss in Mechatronik (Note 1,4) unterstreicht seine solide technische Grundausbildung. Herr Bauer zeichnet sich durch systematisches Denken, Kreativität bei der Lösungsfindung und ausgeprägte Teamfähigkeit aus - Eigenschaften, die für das angestrebte Masterstudium essentiell sind.

    Ich empfehle Herrn Bauer mit Nachdruck für den Master of Science in Robotik. Seine Motivation und sein technisches Verständnis werden ihn zu einem wertvollen Mitglied des Programms machen."

    BEISPIEL 3:
    "Es ist mir eine Freude, Lisa Schmidt für den Master of Science in Künstlicher Intelligenz zu empfehlen. Während ihres Besuchs meines Kurses AI 501: Deep Learning an der RWTH Aachen bewies sie außergewöhnliche Fähigkeiten im Bereich der künstlichen Intelligenz. Frau Schmidt erzielte die Note 1,2 und entwickelte dabei innovative Lösungsansätze für komplexe Deep-Learning-Aufgaben.

    In ihrer Rolle als KI-Entwicklerin bei AI Solutions hat sie ihr Talent bereits unter Beweis gestellt. Ihre selbstständig entwickelten Neural-Network-Architekturen für Bilderkennungssysteme zeigen ihre Fähigkeit, theoretische Konzepte in praktische Anwendungen zu überführen.

    Mit einem Bachelorabschluss in Informatik (Note 1,3) verfügt Frau Schmidt über eine exzellente Grundlage für weiterführende Studien. Ihre Kombination aus technischem Verständnis, Kreativität und analytischem Denken macht sie zu einer vielversprechenden Kandidatin für ein anspruchsvolles Masterprogramm.

    Ich empfehle Frau Schmidt uneingeschränkt für den Master of Science in Künstlicher Intelligenz. Ihre bisherigen Leistungen und ihr Enthusiasmus für das Fachgebiet lassen keinen Zweifel an ihrem zukünftigen Erfolg."

    ANWEISUNGEN:
    1. Erstelle ein neues Empfehlungsschreiben mit ähnlicher Struktur
    2. Ändere:
        - Namen
        - Studienfach
        - Universitäten
        - Kurse
        - Noten
        - aktuelle Berufstätigkeit
    3. Behalte bei:
        - Vier-Absatz-Struktur
        - Professionellen Ton
        - Spezifische Beispiele für Leistungen
        - Klare Empfehlung am Ende

    GEWÜNSCHTES FORMAT:
    - Nur Haupttext
    - Keine Anrede/Grußformel
    - In Markdown formatiert

    MUSTER FÜR ABSÄTZE:
    1. Einleitung mit Empfehlung und Kontext
    2. Spezifische Leistungen und praktische Erfahrung
    3. Akademischer Hintergrund und persönliche Eigenschaften
    4. Abschließende Empfehlung und Zukunftsprognose
    """

In [ ]:
history = [SystemMessage(content=system_prompt)]
interact_with_ai(user_input, history)

<p><font color='black' size="5">
Synthetische Daten generieren
</font></p>

LLMs (Large Language Models) eignen sich zur Erzeugung synthetischer Daten, was besonders nützlich für Testszenarien ist, die realistische Informationen oder eine breite demografische Vielfalt erfordern. Diese Modelle können beispielsweise detaillierte Biografien für verschiedene Berufsgruppen generieren, wodurch realitätsnahe Daten für Simulationen, die Entwicklung von Testalgorithmen oder das Training anderer KI-Systeme bereitgestellt werden.  

Ein möglicher Anwendungsfall wäre die Erstellung synthetischer Biografien für Berufsgruppen wie Softwareentwickler, Kinderkrankenschwestern, Finanzanalysten, Naturwissenschaftslehrer an Universitäten oder Marketingmanager. Jede dieser Biografien könnte individuelle Karrierewege, Qualifikationen und berufliche Erfahrungen umfassen, um vielseitige und praxisnahe Tests zu ermöglichen. Auf diese Weise tragen synthetische Daten zur Verbesserung der Systemleistung bei, ohne auf echte personenbezogene Informationen zurückzugreifen.

In [ ]:
# Anpassung: Abschnitt 2: Konstanten definieren
system_prompt = """
Du bist ein Assistent, der synthetische Daten für eine Person in dem von Ihnen angegebenen Berufsfeld generiert. Erstelle eine kurze Biografie der Person, die nicht länger als 5 Sätze ist. Erwähne die Berufsbezeichnung nicht ausdrücklich.
"""

In [ ]:
# Abschnitt 5: Hauptprogramm
user_input = "Softwareentwickler"

history = [SystemMessage(content=system_prompt)]
interact_with_ai(user_input, history)

Nun wird eine Liste mit mehreren zufällig generierten Biografien erstellt.

In [ ]:
personas = [
"Luftballon-Verkäufer",
"Kinderkrankenschwester",
"Talkmaster",
"Hochschullehrer Wirtschaftswissenschaften",
"Zirkus-Zauberer"
]

In [ ]:
from tqdm import tqdm

# Verwenden Sie tqdm, um den Fortschrittsbalken anzuzeigen
history = [SystemMessage(content=system_prompt)]

for i in tqdm(range(5), desc="Generating Careers"):
    career_choice = personas[i]
    response = interact_with_ai(career_choice, history)
    mprint(response)

# 6 | Textzusammenfassung
---

Große Sprachmodelle (LLMs) wie GPT-4 fassen Texte zusammen, indem sie zentrale Inhalte extrahieren und kompakt wiedergeben. Sie erfassen den Kontext sowie semantische Strukturen und erstellen eine prägnante Version, die die Hauptaussagen erhält. Durch ihre Fähigkeit, verschiedene Textarten zu analysieren – von Fachartikeln bis hin zu narrativen Texten - liefern sie verständliche und relevante Zusammenfassungen. Ihre Flexibilität in Bezug auf Länge und inhaltliche Schwerpunkte macht sie zu einem effizienten Werkzeug für die schnelle Informationsverarbeitung.

<p><font color='black' size="5">
Einzelnes PDF zusammenfassen
</font></p>

Zunächst wird die Zusammenfassung einer einzelnen PDF-Datei betrachtet. LangChang nutzt spezielle Dokumentlader, um verschiedene Dateiformate wie PDFs zu verarbeiten. Für unterschiedliche Datentypen stehen spezifische Lader zur Verfügung. Im folgenden Code wird eine PDF-Datei geladen und mithilfe einer allgemeinen Systemaufforderung analysiert und zusammengefasst.

Zunächst wird das PDF-Dokument *Attention Is All You Need* über den **„PyPDFLoader“** von der angegebenen URL (**https://arxiv.org/pdf/1706.03762**) geladen. Anschliessend wird der Text an das LLM übergeben. Schließlich erfolgt die Ausgabe des zusammengefassten Inhalts im **Markdown-Format**, um die ursprüngliche Formatierung beizubehalten.

In [ ]:
# Abschnitt 1: Importe
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

# Importe für einzelne Zusammenfassung
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain import PromptTemplate

# Importe für mehrere Zusammenfassungen
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document

In [ ]:
# Abschnitt 2: Konstanten definieren
model_name = "gpt-4o-mini"
temperature = 0.0

template = """
Schreibe eine kurze Zusammenfassung der präsentierten Informationen. Schreibe die Zusammenfassung auf Deutsch.

{input}

ZUSAMMENFASSUNG:
"""

In [ ]:
# Abschnitt 3: Chat-Komponenten initialisieren

# Promt-Template erstellen
prompt = PromptTemplate.from_template(template)

# Modell definieren
llm = ChatOpenAI(model=model_name, temperature=temperature)

# Parser erstellen
parser = StrOutputParser()

# Verkettung
chain = prompt | llm | parser

In [ ]:
# Abschnitt 4: Funktionen definieren
def load_document(url: str):
    """Lädt ein Dokument von einer URL und splittet es in Chunks"""
    loader = PyPDFLoader(url)
    return loader.load_and_split()

In [ ]:
# Abschnitt 5: Hauptprogramm
url = "https://arxiv.org/pdf/1706.03762"
docs = load_document(url)
text = " ".join([doc.page_content for doc in docs])

In [ ]:
response = chain.invoke({"input": text})

mprint("## ✨ Zusammenfassung:")
mprint("---")
mprint(response)

<p><font color='black' size="5">
Mehrere PDFs zusammenfassen
</font></p>

Diese vier wissenschaftlichen Arbeiten haben wesentliche Fortschritte im Bereich der generativen KI und der Verarbeitung natürlicher Sprache (NLP) ermöglicht.  

1. **"Attention Is All You Need" (Vaswani et al., 2017)** führte die **Transformer-Architektur** ein, die sich als grundlegendes Modell für moderne NLP-Systeme etabliert hat. Der Verzicht auf rekurrente Strukturen und die Einführung des Selbstaufmerksamkeitsmechanismus ermöglichten effizientere und leistungsfähigere Modelle wie GPT und BERT.  

2. **"BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" (Devlin et al., 2018)** revolutionierte die Kontextverarbeitung in NLP-Modellen durch **bidirektionales Training**, das es ermöglichte, sowohl vorhergehende als auch nachfolgende Wörter gleichzeitig zu analysieren. Dadurch erzielte BERT erhebliche Verbesserungen bei zahlreichen NLP-Aufgaben.  

3. **"Language Models are Few-Shot Learners" (Brown et al., 2020)** untersuchte das Potenzial **großskalierter Transformer-Modelle** am Beispiel von GPT-3. Die Studie zeigte, dass die Leistung solcher Modelle durch Skalierung verbessert wird und sie auch ohne aufgabenspezifisches Training effektive Ergebnisse liefern können.  

4. **"Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer" (Raffel et al., 2019)** präsentierte **T5 (Text-to-Text Transfer Transformer)**, ein Modell, das alle NLP-Aufgaben in ein einheitliches Text-zu-Text-Format überführt. Dadurch wurde das Transferlernen zwischen verschiedenen Aufgaben vereinfacht.  

Diese Arbeiten bilden zusammen die Grundlage für viele moderne NLP-Modelle und deren Anwendungen in generativer KI.

In [ ]:
# Abschnitt 5: Hauptprogramm
urls = [
    "https://arxiv.org/pdf/1706.03762",
    "https://arxiv.org/pdf/1810.04805",
    "https://arxiv.org/pdf/2005.14165",
    "https://arxiv.org/pdf/1910.10683"
]

# Liste für Sammlung der Summaries initialisieren
summaries = []

for url in urls:
    # Dokumente laden und als String formatieren
    print(f"Verarbeite Artikel: {url}")
    docs = load_document(url)
    text = " ".join([doc.page_content for doc in docs])

    # LLM aufrufen und Zusammenfassung erstellen
    response = chain.invoke({"input": text})

    # Zusammenfassungen sammeln
    summaries.append(response)

    # Zusammenfassung ausgeben
    mprint("## ✨ Zusammenfassung:")
    mprint("---")
    mprint(response)
    print()

Nachdem die einzelnen Artikel zusammengefasst wurden, folgt die Erstellung einer übergreifenden Übersicht durch die Kombination dieser Zusammenfassungen. Hierfür werden die Texte zunächst zu einer langen Zeichenfolge zusammengeführt.  

Das Endergebnis wird im **Markdown-Format** ausgegeben, um eine klare Struktur und Lesbarkeit zu gewährleisten. Auf diese Weise entsteht eine prägnante, aber inhaltlich umfassende Synthese der Originalartikel.

In [ ]:
# Gesamtzusammenfassung erstellen
gesamt_text = " ".join(summaries)
response = chain.invoke({"input": gesamt_text})

mprint("## ✨ Gesamt-Zusammenfassung:")
mprint("---")
mprint(response)

# 7 | Textklassifizierung
---

Große Sprachmodelle (LLMs) haben die Textklassifizierung grundlegend verändert, indem sie einen flexibleren und effizienteren Ansatz als herkömmliche maschinelle Lernmethoden bieten. Während traditionelle Verfahren umfangreiche, manuell annotierte Datensätze benötigten, um Modelle zu trainieren, bieten LLMs eine zeitsparende Alternative.  

Durch die **Few-Shot Klassifizierung**, die mit beschriftete Beispiele funktioniert, können LLMs Texte allein auf Basis weniger natürlichsprachlichen Beispiele kategorisieren. Diese Fähigkeit beruht auf dem umfassenden Wissen, das die Modelle während ihres Trainings mit großen Textkorpora erworben haben.  

Dieser Ansatz reduziert den Aufwand für die Einrichtung einer Textklassifizierung erheblich und ermöglicht eine schnelle Anpassung an neue Anforderungen. Dadurch eignen sich LLMs für verschiedene Anwendungsfälle, darunter **Stimmungsanalyse, Themenkategorisierung, Spam-Erkennung und Inhaltsmoderation**.  

Im folgenden Code werden SMS-Texte analysiert. Anschließend erfolgt deren Klassifizierung mithilfe eines LLMs.

Im folgenden Programm erfolgt eine Klassifizierung der SMS-Texte.


In [ ]:
# Abschnitt 1: Importe
import pandas as pd

In [ ]:
# Abschnitt 2: Konstanten definieren
system_prompt = """
Du bist ein Experte für die Klassifikation von SMS-Nachrichten.

Deine Aufgabe:
- Klassifiziere SMS-Nachrichten als 'spam' oder 'ham'
- Gib nur das Klassifikationsergebnis zurück, KEINE weiteren Erläuterungen oder Kommentare!
- Verwende 'sonstiges' wenn keine klare Zuordnung möglich ist.
- Gebe als Antwort NUR "spam" oder "ham" zurück.
- Wenn keine Entscheidung möglich ist dann "sonstiges".
- Gebe keine Begründung.
- Verwende keine Spiegelstriche.

Beachte die Beispiele für die Klassifikation:
- spam: Werbung, Gewinnspiele, kostenpflichtige Dienste
- ham: Normale Kommunikation, persönliche Nachrichten
"""

# Beispiel-SMS für das Training
train_sms = {
    'spam': [
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
        "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
        "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only."
    ],
    'ham': [
        "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",
        "Ok lar... Joking wif u oni...",
        "U dun say so early hor... U c already then say..."
    ]
}

test_sms = [
    "WINNER!! As a valued customer you have won £1000 prize!",
    "Hey, what time is the meeting tomorrow?",
    "Free entry! Win an iPhone now! Click here:",
    "I'll be home in 10 minutes"
]

In [ ]:
model_name = "gpt-4o-mini"
temperature = 0.0

# Modell definieren
llm = ChatOpenAI(model=model_name, temperature=temperature)

# Parser erstellen
parser = StrOutputParser()

# Verkettung
chain = prompt | llm | parser

In [ ]:
# Abschnitt 3: Chat-Komponenten initialisieren
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [ ]:
def create_training_examples() -> str:
    """ Erstellt Trainingsbeispiele aus den Beispiel-SMS """
    examples = []
    for label, messages in train_sms.items():
        for msg in messages:
            examples.append(f"{label}\t{msg}")
    return "\n".join(examples)

def classify(sms_text: str, history: list) -> str:
    """ Klassifiziert eine einzelne SMS-Nachricht """
    # Erstelle Prompt mit Trainingsbeispielen
    full_prompt = f"""
    Klassifiziere die folgende SMS basierend auf diesen Beispielen:

    {create_training_examples()}

    Gebe als Antwort NUR "spam" oder "ham" zurück.
    Wenn keine Entscheidung möglich ist dann "sonstiges".
    Gebe keine Begründung.
    Verwende keine Spiegelstriche.

    Hier ist die SMS:
    {sms_text}
    """
    response = chain.invoke(
    {"system_prompt": system_prompt, "history": history, "input": full_prompt}
)
    return response.strip()

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=system_prompt)]

mprint("## ✨ Klassifiziere Test-SMS:")
mprint("---")

# SMS-Klassifizierung
for i, sms in enumerate(test_sms, 1):
    response = classify(sms, history)
    mprint(f"**Test SMS** #{i}:")
    mprint(sms)
    mprint(f"**Klassifizierung:** {response}")
    print()

<p><font color='black' size="5">
Namen klassifizieren
</font></p>

Die Analyse von Wörtern in Texten stellt für künstliche Intelligenz eine interessante Herausforderung dar, besonders bei der Zero-Shot Klassifizierung ohne Trainingsdaten. Hierbei muss das Modell spontan entscheiden, ob ein Wort ein Name oder ein reguläres Wort ist. Im folgenden Text soll getestet werden, ob eine generative KI in der Lage ist, Wörter korrekt in die Kategorien "Name" oder "Nicht-Name" einzuordnen, ohne zuvor mit ähnlichen Beispielen trainiert worden zu sein. Die zu analysierenden Wörter enthalten sowohl gewöhnliche Begriffe als auch verschiedene Arten von Namen wie Vornamen, Nachnamen oder Firmennamen. Die zentrale Frage lautet: Kann das Modell durch sein grundlegendes Sprachverständnis diese Unterscheidung treffen?

Nun wird ein Large Language Model (LLM) definiert, das den Text analysiert.

Letztendlich wird eine einzelne Eingabeaufforderung genutzt, um diese Namen zu identifizieren und zu kategorisieren.

In [ ]:
# Abschnitt 2: Konstanten definieren
system_prompt = """
Du bist ein präziser Namensextraktor. Deine Aufgabe:
- Extrahiere alle Namen von Menschen, Produkten und Unternehmen aus dem Text
- Gib nur die gefundenen Namen zurück, einer pro Zeile
- Keine zusätzlichen Erklärungen oder Formatierungen
- Keine Duplikate
- Halte Dich konsequent an die Formulierung
"""

user_input = """
Anna und Sarah sind seit dem Studium befreundet. Nach ihrem Abschluss hatten sie ein Vorstellungsgespräch bei Frau Weber für eine Stelle bei der Technovision GmbH, deren Hauptprodukt Futurtech heißt und von der Programmiererin Maria Schmidt entwickelt wurde. Nach dem erfolgreichen Gespräch trafen sie sich mit ihren Freunden Rafael und Aida zum Feiern. An ihrem ersten Arbeitstag lernten sie drei weitere Mitarbeiter kennen: Richard, Lisa und Pia. Später kam noch Matthias Müller zum Team dazu.
"""

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=system_prompt)]

# Namen extrahieren und anzeigen
response = chain.invoke(
    {"system_prompt": system_prompt, "history": history, "input": user_input})

# Ausgabe
mprint("## 📛 Gefundene Namen:")
mprint("---")
mprint(response)

# 8 | LLM schreibt ein Buch
---

**Dieser** Abschnitt zeigt, wie ein Large Language Model (LLM) beim Schreiben eines Buches genutzt werden kann. Da ein Buch eine komplexe, langfristige Aufgabe darstellt, erfolgt der Prozess schrittweise.  

Zunächst wird ein **Thema** gewählt, auf dessen Basis das LLM einen Titel und eine kurze **Zusammenfassung** erstellt. Diese Zusammenfassung dient als inhaltlicher Leitfaden. Anschließend wird ein **Inhaltsverzeichnis** generiert, das die Kapitel und Abschnitte strukturiert.  

Jedes **Kapitel** wird in separaten Sitzungen mit dem LLM ausgearbeitet, wobei die zuvor erstellte Gliederung zur Orientierung dient. Dieser iterative Ansatz gewährleistet Konsistenz und ermöglicht eine detaillierte Ausarbeitung.  

Durch eine methodische Vorgehensweise verwandelt sich eine anfängliche Idee in ein gut strukturiertes Buch. Für kreative Ergebnisse wird ein Sprachmodell mit einer Temperatur von 0,7 verwendet.

In [ ]:
# Die Zeile installiert still und automatisch das Programm wkhtmltopdf, das HTML in PDF umwandelt.
# Alle Ausgaben und Fehler werden dabei unterdrückt.
!sudo apt-get install -qq -y wkhtmltopdf > /dev/null 2>&1

In [ ]:
# Abschnitt 1: Importe
import markdown
import pdfkit

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
# Abschnitt 2: Konstanten definieren
system_prompt = """
Du bist ein kreativer Autor und Schriftsteller.

Deine Texte sind:
- Fesselnd und gut strukturiert
- Reich an Details und Beschreibungen
- Konsistent im Stil und der Erzählweise
- Mit klarer Handlung und Charakterentwicklung
- In ansprechendem Deutsch verfasst

Bei deinem Schreiben:
- Entwickelst du überzeugende Charaktere
- Erschaffst atmosphärische Szenen
- Behältst den roten Faden bei
- Verwendest kreative Sprachbilder
- Achtest auf Spannung und Dynamik
"""

num_chapters = 13
subject = "Sci-Fi Fantasy Thriller"

In [ ]:
# Abschnitt 3: Chat-Komponenten initialisieren
model_name = "gpt-4o-mini"
temperature = 0.7

llm = ChatOpenAI(model=model_name, temperature=temperature)

parser = StrOutputParser()

chain = prompt | llm | parser

Einfache Hilfsfunktionen werden implementiert, um das LLM mit einer benutzerdefinierten Systemaufforderung zu befragen. Diese Aufforderung informiert das Modell darüber, dass es beim Schreiben eines Buches assistiert.


<p><font color='black' size="5">
Titel, Synopse, Inhaltsverzeichnis
</font></p>

Für dieses Buch kann der Benutzer das Thema über die Variable `SUBJECT` festlegen. Anschließend wird das LLM angewiesen, einen zufälligen Titel basierend auf diesem Thema zu generieren. Dabei wird darauf geachtet, dass die Eingabeaufforderung präzise formuliert ist, um zu vermeiden, dass das LLM zusätzliche Einleitungen wie „Hier ist ein zufälliger Titel“ hinzufügt.

<p><font color='black' size="5">
Titel
</font></p>

In [ ]:
title_prompt = ChatPromptTemplate.from_template(
    "Gebe mir einen zufälligen Titel für ein Buch zum Thema {subject}. "
    "Gibt nur den Titel zurück, keinen zusätzlichen Text."
)

formatted_prompt = title_prompt.format_messages(subject=subject)

In [ ]:
response_titel = chain.invoke(
    {"system_prompt": system_prompt, "history": [], "input": formatted_prompt})

mprint("## ✨ Titel:")
mprint("---")
mprint(response_titel)

<p><font color='black' size="5">
Synopsis
</font></p>

Nachdem der Titel festgelegt wurde, kann nun eine zufällige Zusammenfassung des Buches generiert werden. Diese gibt eine erste inhaltliche Orientierung und dient als Grundlage für die weitere Strukturierung.

In [ ]:
synopsis_prompt = ChatPromptTemplate.from_template(
    "Gebe mir eine Inhaltsangabe für ein Buch zum Thema {subject}. "
    "Gebe nur die Inhaltsangabe zurück, keinen zusätzlichen Text."
)

formatted_prompt = synopsis_prompt.format_messages(subject=response_titel)

In [ ]:
response_synopsis = chain.invoke(
    {"system_prompt": system_prompt, "history": [], "input": formatted_prompt})

mprint("## ✨ Synopsis:")
mprint("---")
mprint(response_synopsis)

<p><font color='black' size="5">
Inhaltsverzeichnis
</font></p>

Nun wird das Inhaltsverzeichnis erstellt, wobei alle zuvor generierten Informationen berücksichtigt werden. Ein spezifisches Format wird vorgegeben, um eine klare Struktur zu gewährleisten. Obwohl die Kapitelnummern leicht ableitbar wären, werden sie explizit angefordert, da das LLM sie ohnehin bereitstellen möchte. Der Ansatz erleichtert eine konsistente Struktur und ermöglicht es, die Nummerierung später gezielt zu entfernen, falls erforderlich.

In [ ]:
toc_prompt = ChatPromptTemplate.from_template(
    "Gebe mir ein Inhaltsverzeichnis für ein Buch mit dem Titel {title} "
    "für ein Buch zum Thema {subject}. Die Buchzusammenfassung lautet {synopsis}. "
    "Gebe das Inhaltsverzeichnis als liste von Kapitelüberschriften zurück. "
    "Erstelle nicht mehr als {num_chapters} Kapitel. "
    "Trenne Kapitelnummer und Kapitelüberschrift mit einem Pipe-Zeichen '|'. "
    "Gebe nur die Kapitelnamen zurück, keinen zusätzlichen Text."
)

formatted_prompt = toc_prompt.format_messages(title=response_titel,
                                              subject=subject,
                                              synopsis=response_synopsis,
                                              num_chapters=num_chapters)

In [ ]:
response_toc = chain.invoke(
    {"system_prompt": system_prompt, "history": [], "input": formatted_prompt})

mprint("## ✨ Inhaltsverzeichnis:")
mprint("---")
mprint(response_toc)


<p><font color='black' size="5">
Erstellen der Kapitel des Buches
</font></p>


Nun wird eine Funktion erstellt, die den Text eines Kapitels generiert. Damit das LLM über ausreichend Kontext verfügt, werden die Zusammenfassung, das Inhaltsverzeichnis und die entsprechende Kapitelnummer übergeben.

In [ ]:
book_prompt = ChatPromptTemplate.from_template(
    "Schreibe Kapitel {chapter_num} mit dem Titel {chapter_title} für ein Buch "
    "mit dem Titel {title} zum Thema {subject}. Die Buchzusammenfassung lautet {synopsis}. "
    "Das Inhaltsverzeichnis lautet {toc}. "
    "Gebe nur den Kapiteltext zurück, keine Kapitelüberschrift, keinen Kapiteltitel, "
    "keine Kapitelnummer, keinen zusätzlichen Text."
)

In [ ]:
# Erstellen eine Liste mit den Kapiteln
titel_liste = [zeile.split('|', 1)[1].strip() for zeile in response_toc.strip().split('\n')]

book_content = f"# {response_titel}"

mprint(f"## ✨ Erstelle Inhalt für {response_titel}:")
mprint("---")

for chapter_num, chapter_title in enumerate(titel_liste, 1):
    formatted_prompt = book_prompt.format_messages(chapter_num=chapter_num,
                                                chapter_title=chapter_title,
                                                title=response_titel,
                                                subject=subject,
                                                synopsis=response_synopsis,
                                                toc=response_toc)

    response_content = chain.invoke(
    {"system_prompt": system_prompt, "history": [], "input": formatted_prompt})

    book_content += f"\n## {chapter_title}\n{response_content}"
    mprint(f"{chapter_num} | {chapter_title}")

<p><font color='black' size="5">
PDF-Datei erstellen
</font></p>

In [ ]:
import markdown2
from weasyprint import HTML, CSS

In [ ]:
# Markdown → HTML
html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <style>
        @page {{
            size: A4;
            margin: 25mm 20mm 30mm 20mm;

            @bottom-center {{
                content: "Seite " counter(page) " von " counter(pages);
                font-size: 10pt;
                font-family: DejaVu Sans, sans-serif;
            }}
        }}
        body {{
            font-family: "DejaVu Sans", sans-serif;
            font-size: 12pt;
        }}
    </style>
</head>
<body>
{markdown2.markdown(book_content)}
</body>
</html>
"""

# PDF erzeugen
HTML(string=html).write_pdf("my_book.pdf")

Das generierte Buch steht nun zum Download bereit.

In [ ]:
from google.colab import files
files.download("my_book.pdf")

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Sentiment-Analyse von Produktbewertungen mit LLM Zero-Shot-Klassifizierung
</font></p>

**Aufgabenstellung**
Entwickeln Sie ein Python-Programm, das mithilfe eines Large Language Models (LLM) Produktbewertungen in die Kategorien "Positiv", "Neutral" oder "Negativ" einordnet und zusätzlich die betroffenen Produktaspekte (Qualität, Preis, Lieferung, Service) identifiziert.

**Lernziele**
Nach Abschluss dieser Übung können Sie:
- Zero-Shot-Klassifizierung für mehrere Kategorien implementieren
- Aspekt-basierte Sentiment-Analyse durchführen
- Komplexe Prompts für Mehrfachklassifizierung erstellen
- Strukturierte Ausgaben aus LLM-Antworten generieren

**Aufgabendetails**

```python
# Beispieldaten
reviews = [
    "Die Qualität des Produkts ist hervorragend, allerdings finde ich den Preis zu hoch.",
    "Schnelle Lieferung, guter Service, faire Preise - besser geht es nicht!",
    "Nach zwei Wochen ging das Gerät kaputt. Der Kundenservice war bei der Reklamation leider keine Hilfe.",
    "Durchschnittliche Qualität, erfüllt seinen Zweck. Lieferung dauerte etwas länger als angegeben."
]
```

**Beispiel-Prompt**
```
Analysiere die folgende Produktbewertung hinsichtlich:
1. Gesamtsentiment (Positiv/Neutral/Negativ)
2. Erwähnte Produktaspekte (Qualität/Preis/Lieferung/Service)
3. Sentiment pro Aspekt

Bewertung: {review}

Antworte im Format:
Gesamtsentiment: [Kategorie]
Identifizierte Aspekte:
- [Aspekt]: [Sentiment]
Begründung: [Analyse]
```